In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# ОЧЕНЬ ПРОСТЫЕ ML БИБЛИОТЕКИ
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  # Самая простая модель
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Настройка графиков
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Готово! Библиотеки загружены")
print("   Используем только LinearRegression - самую простую модель")

✅ Готово! Библиотеки загружены
   Используем только LinearRegression - самую простую модель
